In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import torch
from torch import autocast
from diffusers import StableDiffusionPipeline, DDIMScheduler
from IPython.display import display
from tqdm import tqdm
import json

now_dir_path = os.getcwd()
DAPB_path = os.path.dirname(now_dir_path)
print(f"DAPB_path: {DAPB_path}")
print(f"now_dir_path: {now_dir_path}")
trained_model_path = os.path.join(now_dir_path, "train_outputs")
gen_outputs_path = os.path.join(now_dir_path, "gen_outputs")

# If you want to use previously trained model saved in gdrive, replace this with the full path of model in gdrive
taskname = "MultiATK_SD21_VGGFace2_r8_idx10_1idmatk"
spwords_file = "spwords.json"
totalsteps = 10000
save_path = os.path.join(gen_outputs_path, taskname)
if not os.path.exists(save_path):
    os.makedirs(save_path)
model_path = os.path.join(trained_model_path, taskname, str(totalsteps))
print(f"model_path: {model_path}")
pipe = StableDiffusionPipeline.from_pretrained(model_path, safety_checker=None, torch_dtype=torch.float16).to("cuda")
pipe.scheduler = DDIMScheduler.from_config(pipe.scheduler.config)
pipe.enable_xformers_memory_efficient_attention()
g_cuda = None
#@markdown Can set random seed here for reproducibility.
g_cuda = torch.Generator(device='cuda')
seed = 52362 #@param {type:"number"}
g_cuda.manual_seed(seed)

/data/home/yekai/conda_envs/Metacloakp/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-06-30 21:38:13.450784: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-30 21:38:13.500309: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-06-30 21:38:14.163227: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: C

DAPB_path: /data/home/yekai/github/DiffAdvPerturbationBench
now_dir_path: /data/home/yekai/github/DiffAdvPerturbationBench/multi-dreambooth
model_path: /data/home/yekai/github/DiffAdvPerturbationBench/multi-dreambooth/train_outputs/MultiATK_SD21_VGGFace2_r8_idx10_1idmatk/10000


/data/home/yekai/conda_envs/Metacloakp/lib/python3.10/site-packages/diffusers/models/modeling_utils.py:101: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(c

In [ ]:
special_text = []
with open(f"{now_dir_path}/{spwords_file}", "r") as f:
    special_text = json.load(f)["spwords_list"]
# special_text = [
#     "vux", "jiq", "qem", "zod", "dul", "wex", "hob", "taf", "yib", "nuz",
#     "gax", "fep", "cim", "pyk", "teb", "lom", "sir", "dap", "kex", "yoc",
#     "mib", "zow", "ruk", "hif", "cun", "baj", "tox", "geq", "vyx", "qum",
#     "rek", "sov", "hax", "zud", "dif", "koy", "wen", "jib", "rax", "miv",
#     "pob", "lut", "seg", "yox", "cud", "vig", "bez", "nam", "foj", "xur"
# ]
# special_text = [
#      "vux", "jiq", "qem", "zod", "dul", "wex", "hob", "taf", "yib", "nuz",
# ]
# save_path = f"/data/home/yekai/github/sampleall/multi-dreambooth/gen_output/{taskname}"
# if not os.path.exists(save_path):
#     os.mkdir(save_path)
# instance_num = len(special_text)
instance_num = 10
prompt_list = [f"a photo of {special_text[i]} person" for i in range(instance_num)]
# prompt_list = [f"a noised photo of {special_text[i]} person" for i in range(instance_num)] #@param {type:"string"}
# prompt_list = [f"a noised photo of person" for i in range(instance_num)]
negative_prompt = "" #@param {type:"string"}
num_samples = 16 #@param {type:"number"}
guidance_scale = 7.5 #@param {type:"number"}
num_inference_steps = 100 #@param {type:"number"}
height = 512 #@param {type:"number"}
width = 512 #@param {type:"number"}
for i in range(instance_num):
    with autocast("cuda"), torch.inference_mode():
        images = pipe(
            prompt_list[i],
            height=height,
            width=width,
            negative_prompt=negative_prompt,
            num_images_per_prompt=num_samples,
            num_inference_steps=num_inference_steps,
            guidance_scale=guidance_scale,
            generator=g_cuda
        ).images
    if not os.path.exists(f"{save_path}/{i}"):
        os.mkdir(f"{save_path}/{i}")
    for index, img in enumerate(images):
        img.save(f"{save_path}/{i}/{index}.jpg")

100%|██████████| 100/100 [00:31<00:00,  3.18it/s]


#### 评估生成图片

In [ ]:

import os
import sys
import torch
sys.path.append("/data/home/yekai/github/MetaCloak")
sys.path.append("/data/home/yekai/github/MetaCloak/robust_facecloak")


from eval_score import get_score
import numpy as np
from robust_facecloak.attacks.worker.differential_color_functions import rgb2lab_diff, ciede2000_diff
from robust_facecloak.generic.data_utils import PromptDataset, load_data_by_picname

def find_max_pixel_change(original_img, noisy_img):
    diff = torch.abs(original_img - noisy_img)
    
    # Find the maximum pixel difference
    max_change = torch.max(diff)
    
    return max_change.item()

def get_L0(original_img, noisy_img):
    diff = torch.abs(original_img - noisy_img)
    diff_L0 = torch.sum(diff > 0, dim=(1, 2, 3))
    # Find the maximum pixel difference
    mean_L0 = torch.mean(diff_L0.float())
    return mean_L0.item()

def get_L1(original_img, noisy_img):
    diff = torch.abs(original_img - noisy_img)
    diff_L1 = torch.sum(diff, dim=(1, 2, 3))
    mean_L1 = torch.mean(diff_L1.float())
    return mean_L1.item()

def get_change_p(original_img, noisy_img):
    diff = torch.abs(original_img - noisy_img)
    diff_L0_all = torch.sum(diff > 0, dim=(0, 1, 2, 3))
    pix_num_all = original_img.shape[0] * original_img.shape[1] * original_img.shape[2] * original_img.shape[3]
    change_p = diff_L0_all / pix_num_all
    return change_p.item()

def get_ciede2000_diff(ori_imgs,advimgs):
    device = torch.device('cuda')
    ori_imgs_0_1 = ori_imgs/255
    advimgs_0_1 = advimgs/255
    advimgs_0_1.clamp_(0,1)
    # print(f'ori_imgs_0_1.min:{ori_imgs_0_1.min()}, ori_imgs_0_1.max:{ori_imgs_0_1.max()}')
    # print(f'advimgs_0_1.min:{advimgs_0_1.min()}, advimgs_0_1.max:{advimgs_0_1.max()}')
    X_ori_LAB = rgb2lab_diff(ori_imgs_0_1,device)
    advimgs_LAB = rgb2lab_diff(advimgs_0_1,device)
    # print(f'advimgs: {advimgs}')
    # print(f'ori_imgs: {ori_imgs}')
    color_distance_map=ciede2000_diff(X_ori_LAB,advimgs_LAB,device)
    # print(color_distance_map)
    scores = torch.norm(color_distance_map.view(ori_imgs.shape[0],-1),dim=1)
    # print(f'scores: {scores}')
    # mean_scores = torch.mean(scores)
    # 100
    return torch.mean(scores)






In [ ]:
class OrderedSet:
    def __init__(self):
        self._data = dict()

    def add(self, item):
        self._data[item] = None

    def __contains__(self, item):
        return item in self._data

    def __iter__(self):
        return iter(self._data)

    def __len__(self):
        return len(self._data)

    def __repr__(self):
        return f"OrderedSet({list(self._data)})"

    def discard(self, item):
        self._data.pop(item, None)

    def clear(self):
        self._data.clear()


In [ ]:
ori_data_path = "/data/home/yekai/github/MetaCloak/dataset/VGGFace2-clean"
clean_ref_dir_list = [os.path.join(ori_data_path,str(i),'set_C') for i in range(instance_num)]
ori_pics_dir_list = [os.path.join(ori_data_path,str(i),'set_B') for i in range(instance_num)]
# noisy_pics_dir = save_path


noisy_pics_dir_list = [os.path.join("/data/home/yekai/github/MetaCloak/dataset/VGGFace2-MetaCloak480",str(i),'set_B') for i in range(instance_num)]
gen_pics_dir_list = [os.path.join(save_path,str(i)) for i in range(instance_num)]
# clean_ref_dir = ori_pics_dir
results = []
key_set = OrderedSet()
for i in tqdm(range(instance_num)):
    score_dict = get_score(gen_pics_dir_list[i],clean_ref_dir_list[i])
    # print(score_dict)
    k_list = list(score_dict.keys())
    tmp_score = []
    for k in k_list:
        key_set.add(k)
        means = []
        means.append(score_dict[k])
        # print(f"{k}_mean {np.mean(means)}")
        tmp_score.append(np.mean(means))
        stds = []
        stds.append(score_dict[k])
        # print(f"{k}_std {np.std(stds)}")



    original_data = load_data_by_picname(ori_pics_dir_list[i])
    perturbed_data = load_data_by_picname(noisy_pics_dir_list[i])

    # print(original_data-perturbed_data)
    # print(perturbed_data)

    # print(original_data[0][0])
    # print(perturbed_data[0][0])

    max_noise_r = find_max_pixel_change(perturbed_data, original_data)
    noise_L0 = get_L0(perturbed_data, original_data)
    noise_L1 = get_L1(perturbed_data, original_data)
    noise_p = get_change_p(perturbed_data, original_data)
    ciede2000_score = get_ciede2000_diff(original_data, perturbed_data)
    
    score_dict['max_noise_r'] = max_noise_r
    score_dict['noise_L0'] = noise_L0
    score_dict['pix_change_mean'] = noise_L1/(512*512)/2
    score_dict['change_area_mean'] = noise_p*100
    score_dict['ciede2000_score'] = ciede2000_score
    tmp_score.append(max_noise_r)
    tmp_score.append(noise_L0)
    tmp_score.append(noise_L1)
    tmp_score.append(noise_p*100)
    tmp_score.append(float(ciede2000_score))
    key_set.add('max_noise_r')
    key_set.add('noise_L0')
    key_set.add('pix_change_mean')
    key_set.add('change_area_mean')
    key_set.add('ciede2000_score')
    results.append(tmp_score)
    # break

    # print(f"max_noise_r {max_noise_r:.2f}")
    # print(f"noise_L0 {noise_L0:.2f}")
    # print(f"pix_change_mean {noise_L1:.2f}")
    # print(f"change_area_mean {noise_p*100:.2f}")
    # print(f"ciede2000_score {ciede2000_score:.2f}")

    # print(score_dict)

In [ ]:
import pandas as pd

# 假设 key_set 是列名，results 是二维列表（每行是一个 tmp_score）
df = pd.DataFrame(results, columns=list(key_set))

# 打印保留三位小数
print(df.round(3).to_string(index=False))

In [ ]:
df.to_csv(f"/data/home/yekai/github/sampleall/multi-dreambooth/gen_output/{taskname}.csv", index=True)